In [4]:
# import the neccessary libs
from matplotlib.patches import ConnectionPatch
import matplotlib.pyplot as plt
import numpy as np 
import scipy.spatial.distance as dist

* Speech is a MultiVariate

In [5]:
from scipy.io import wavfile
import IPython.display as ipyd
import librosa
import librosa.display